In [1]:
# Libraries

import numpy
import os
import platform
import random
import shutil
import sys

In [2]:
# Ensure source path

ROOT = os.getcwd()

while not ROOT.endswith('upolanc-thesis') :
	ROOT = os.path.abspath(os.path.join(ROOT, os.pardir))

	if len(ROOT) < len('upolanc-thesis') :
		if   platform.system() == 'Linux' :
			ROOT = '/d/hpc/home/up4472/workspace/upolanc-thesis'
		elif platform.system() == 'Windows' :
			ROOT = 'C:\\Developer\\Workspace\\PyCharm\\Projects\\upolanc-thesis'
		else :
			raise ValueError()

		print(f'Warning : could not find correct directory, using default : {ROOT}')
		break

if ROOT not in sys.path :
	sys.path.append(ROOT)

os.chdir(ROOT)

In [3]:
# Code

from source.python.data.mutation import mutation_sequence
from source.python.data.feature  import feature_extractor
from source.python.io            import loader
from source.python.io            import writer
from source.python               import runtime

runtime.set_numpy_format()
runtime.set_pandas_format()
runtime.set_plot_theme()

# 1. Setup

In [4]:
# Setup some directory paths

FILTER_ID = 1
SUBFOLDER = 'filter' + str(FILTER_ID)

CWD = ROOT
OUT = os.path.join(CWD, 'output')
RES = os.path.join(CWD, 'resources')

OUT_DATA   = os.path.join(OUT, 'nbp04-feature', SUBFOLDER)
RES_GENOME = os.path.join(RES, 'genome')
RES_NBP01  = os.path.join(OUT, 'nbp01-filter',  SUBFOLDER)
RES_NBP02  = os.path.join(OUT, 'nbp02-anndata', SUBFOLDER)

shutil.rmtree(OUT_DATA, ignore_errors = True)

os.makedirs(OUT_DATA, exist_ok = True)

print(f'     Root Directory : {CWD}')
print(f'   Output Directory : {OUT_DATA}')
print(f' Resource Directory : {RES_GENOME}')
print(f' Resource Directory : {RES_NBP01}')

     Root Directory : C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis
   Output Directory : C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\output\nbp04-feature\filter1
 Resource Directory : C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\resources\genome
 Resource Directory : C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\output\nbp01-filter\filter1


In [5]:
# Load the annotated and cleaned data

gene_assembly = loader.load_faidx(
	filename  = os.path.join(RES_GENOME, 'arabidopsis-r36', 'gene-assembly.fa')
)

gene_annotation = loader.load_csv(
	filename   = os.path.join(RES_NBP01, 'gene-annotation.csv'),
	low_memory = False
)

anndata = loader.load_h5ad(
	filename = os.path.join(RES_NBP02, 'arabidopsis-r36.h5ad')
)

filter_dict = loader.load_json(
	filename = os.path.join(RES_NBP01, 'filter.json')
)

In [6]:
# Filtered transcripts

keep_transcript = filter_dict['data']['keep_transcript']
drop_transcript = filter_dict['data']['drop_transcript']

In [7]:
# Define the region lengths

lengths = {
	'prom_full' : [int( 5000), int(  0)],
	'prom'      :  int( 1000),
	'utr5'      :  int(  300),
	'cds'       :  int(10000),
	'utr3'      :  int(  350),
	'term'      :  int(  500),
	'term_full' : [int(    0), int(500)]
}

padding = {
	'prom_full' : 'left',
	'prom'      : 'left',
	'utr5'      : 'left',
	'cds'       : 'none',
	'utr3'      : 'left',
	'term'      : 'right',
	'term_full' : 'right'
}

# 2. Transcript Regions

In [8]:
# Group annotations into regions

regions = feature_extractor.annotation_to_regions(
	annotation = gene_annotation,
	lengths    = lengths
)

print()
print('Gene       : {:5d} / {:5d}'.format(regions['Gene'].nunique(), gene_annotation['Gene'].nunique()))
print('Transcript : {:5d} / {:5d}'.format(regions['Transcript'].nunique(), gene_annotation['Transcript'].nunique()))

Passed 1st assertion : True
Passed 2nd assertion : True

Gene       : 22579 / 27420
Transcript : 22579 / 27420


# 3. Transcript Features

In [9]:
# Convert the regions into sequences and features

sequences, features = feature_extractor.regions_to_features(
	faidx     = gene_assembly,
	dataframe = regions,
	lengths   = lengths,
	verbose   = False
)

In [10]:
# Convert the dataframe into a dictionary for ease of use

sequences = sequences.copy()
sequences = sequences.set_index('Transcript', drop = False)
sequences = sequences.rename_axis(None, axis = 'index')
sequences = sequences.to_dict('index')

In [11]:
# Convert the dataframe into a dictionary for ease of use

features = features.copy()
features = features.set_index('Transcript', drop = False)
features = features.rename_axis(None, axis = 'index')
features = features.to_dict('index')

In [12]:
# Split to keep and drop

keep_sequences = {k : v for k, v in sequences.items() if k in keep_transcript}
keep_features  = {k : v for k, v in features.items()  if k in keep_transcript}

drop_sequences = {k : v for k, v in sequences.items() if k in drop_transcript}
drop_features  = {k : v for k, v in features.items()  if k in drop_transcript}

## 3.1 Fasta

In [13]:
# Add a header field with more transcript information

keep_sequences = feature_extractor.sequences_extend_kvpair(
	sequences = keep_sequences,
	regions   = regions,
	header    = '{} | {} | {}:{}-{} | {}'
)

drop_sequences = feature_extractor.sequences_extend_kvpair(
	sequences = drop_sequences,
	regions   = regions,
	header    = '{} | {} | {}:{}-{} | {}'
)

In [14]:
# Example of a padded strand

transcript = list(keep_sequences.keys())[0]

feature_extractor.print_extracted_sequence(
	transcript = transcript,
	sequences  = keep_sequences,
	space      = True
)

AT1G01010.1 | + | 1:1-3630 | 3630
CCCTAAACCC TAAACCCTAA ACCCTAAACC TCTGAATCCT TAATCCCTAA ATCCCTAAAT CTTTAAATCC TACATCCATG AATCCCTAAA TACCTAATTC 
CCTAAACCCG AAACCGGTTT CTCTGGTTGA AAATCATTGT GTATATAATG ATAATTTTAT CGTTTTTATG TAATTGCTTA TTGTTGTGTG TAGATTTTTT 
AAAAATATCA TTTGAGGTCA ATACAAATCC TATTTCTTGT GGTTTTCTTT CCTTCACTTA GCTATGGATG GTTTATCTTC ATTTGTTATA TTGGATACAA 
GCTTTGCTAC GATCTACATT TGGGAATGTG AGTCTCTTAT TGTAACCTTA GGGTTGGTTT ATCTCAAGAA TCTTATTAAT TGTTTGGACT GTTTATGTTT 
GGACATTTAT TGTCATTCTT ACTCCTTTGT GGAAATGTTT GTTCTATCAA TTTATCTTTT GTGGGAAAAT TATTTAGTTG TAGGGATGAA GTCTTTCTTC 
GTTGTTGTTA CGCTTGTCAT CTCATCTCTC AATGATATGG GATGGTCCTT TAGCATTTAT TCTGAAGTTC TTCTGCTTGA TGATTTTATC CTTAGCCAAA 
AGGATTGGTG GTTTGAAGAC ACATCATATC AAAAAAGCTA TCGCCTCGAC GATGCTCTAT TTCTATCCTT GTAGCACACA TTTTGGCACT CAAAAAAGTA 
TTTTTAGATG TTTGTTTTGC TTCTTTGAAG TAGTTTCTCT TTGCAAAATT CCTCTTTTTT TAGAGTGATT TGGATGATTC AAGACTTCTC GGTACTGCAA 
AGTTCTTCCG CCTGATTAAT TATCCATTTT ACCTTTGTCG TAGATATTAG GTAATCTGTA AGTCAACTCA T

In [15]:
# Example of a padded strand

transcript = list(drop_sequences.keys())[0]

feature_extractor.print_extracted_sequence(
	transcript = transcript,
	sequences  = drop_sequences,
	space      = True
)

AT1G01060.6 | - | 1:37870-42869 | 5000
ACTATTAAGT TGTTCATGCA TGTATAAGAT AAAATTATTA TATATTGATG CATTTAATGA GTTAAATAAT TTATAAACAA TGTAATAAAT TACATTTTAT 
AAGTTGTAAG AAAGATCAAT AGTTGTAGCT AGTTGGTAAG AAAAAATATT CTGTTAATTT AACAAGTACA AATTTGCTCG ATTTATAGTT TTGTAATATC 
AGGAAAACAA AATTTCAACT TCAAGTTCTA TATTTTCAGT GTTCTAAATC CAAACTATAT TTGAAGGAGT AAACTAACAT TAGTTTTAAC TTCAAAAAGA 
TACTTGTCTA GATAGAAAAA AACGTTTAGA CTTCTTAGAG CATCATTATC AATAGGTTGT TAACAATATC ATAAGAACTA TTAAAATATT ATATTAAACA 
AAATAAGAAT AAAAAAAAAT AAAAAAGTGT GCTTGAGATG AAAAATGGCT CTTAGCTAAG AGTTTTCTTC CTTATTCTCT TCTTTTGTAA GAGCCTTTCT 
AATATTTAAA CATTATGTTA TATATCAAAA TTGTTTAGAA CTACATGTAA GAGTCCATTG ATAAAGATGG TCTTAGCCCA CTTAAAGTAA CTCATGAACA 
TAGAGAAAAT CAGTCTAACT TCTAAAATCA AGACGTGTAT GAAAGGCTTA GACACTTGTT TCTGAAAAAT TATGTCAAAC CAACTCTGAA GCCATGCATA 
AGTAGGCATA TCCCAAATTA ATTCCACTTT CACGACCACT CATGCACTCG AACTCAGATA GCAATACCAA CTCCTTTTTG TCTTTTCCTT CTTTTTTTCG 
TATCACAAAA GTTTTGTCAA GTTCAATCTA GATAGATTGC GATCAATCTT ATCAATGGAA ATAGTAA

In [16]:
# Save the transcript region sequences

tuples = [
	('Prom_Full', 'promoter-full'),
	('Prom',      'promoter'),
	('UTR5',      'utr5'),
	('CDS',       'cds'),
	('UTR3',      'utr3'),
	('Term',      'terminator'),
	('Term_Full', 'terminator-full')
]

for region, filename in tuples :
	writer.write_fasta(
		data = {
			item[region]['key'] : item[region]['seq']
			for item in keep_sequences.values()
		},
		filename = os.path.join(OUT_DATA, f'sequences-{filename}-keep.fasta')
	)

for region, filename in tuples :
	writer.write_fasta(
		data = {
			item[region]['key'] : item[region]['seq']
			for item in drop_sequences.values()
		},
		filename = os.path.join(OUT_DATA, f'sequences-{filename}-drop.fasta')
	)

## 3.2 Sequences

In [17]:
# Merge transcript regions and pad accordingly

keep_2150, keep_6150 = feature_extractor.merge_and_pad_sequences(
	sequences = keep_sequences,
	lengths   = lengths,
	padding   = padding
)

drop_2150, drop_6150 = feature_extractor.merge_and_pad_sequences(
	sequences = drop_sequences,
	lengths   = lengths,
	padding   = padding
)

In [18]:
# Display an example of a merged transcript sequence

transcript = list(keep_2150.keys())[0].split(' | ')[0]

feature_extractor.print_padded_sequence(
	transcript = transcript,
	sequences  = keep_2150,
	space      = True
)

AT1G01010.1 | + | 1:2631-6399 | 2150
ATATTGCTAT TTCTGCCAAT ATTAAAACTT CACTTAGGAA GACTTGAACC TACCACACGT TAGTGACTAA TGAGAGCCAC TAGATAATTG CATGCATCCC 
ACACTAGTAC TAATTTTCTA GGGATATTAG AGTTTTCTAA TCACCTACTT CCTACTATGT GTATGTTATC TACTGGCGTG GATGCTTTTA AAGATGTTAC 
GTTATTATTT TGTTCGGTTT GGAAAACGGC TCAATCGTTA TGAGTTCGTA AGACACATAC ATTGTTCCAT GATAAAATGC AACCCCACGA ACCATTTGCG 
ACAAGCAAAA CAACATGGTC AAAATTAAAA GCTAACAATT AGCCAGCGAT TCAAAAAGTC AACCTTCTAG ATGGATTTAA CAACATATCG ATAGGATTCA 
AGATTAAAAA TAAGCACACT CTTATTAATG TTAAAAAACG AATGAGATGA AAATATTTGG CGTGTTCACA CACATAATCT AGAAGACAGA TTCGAGTTGC 
TCTCCTTTGT TTTGCTTTGG GAGGGACCCA TTATTACCGC CCAGCAGCTT CCCAGCCTTC CTTTATAAGG CTTAATTTAT ATTTATTTAA ATTTTATATG 
TTCTTCTATT ATAATACTAA AAGGGGAATA CAAATTTCTA CAGAGGATGA TATTCAATCC ACGGTTCACC CAAACCGATT TTATAAAATT TATTATTAAA 
TCTTTTTTAA TTGTTAAATT GGTTTAAATC TGAACTCTGT TTACTTACAT TGATTAAAAT TCTAAACCAT CATAAGTAAA AAATAATATG ATTAAGACTA 
ATAAATCTTA ATAGTTAATA CTACTCGGTT TACTACATGA AATTTCATAC CATCAATTGT TTTAATAAT

In [19]:
# Display an example of a merged transcript sequence

transcript = list(keep_6150.keys())[0].split(' | ')[0]

feature_extractor.print_padded_sequence(
	transcript = transcript,
	sequences  = keep_6150,
	space      = True
)

AT1G01010.1 | + | 1:1-6399 | 6150
---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- 
---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- 
---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- 
---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- 
---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- 
---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- 
---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- 
---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- ---------- 
---------- ---------- ---------- ---------- ---------- ---------- ---------- -

In [20]:
# Save the transcript sequences

writer.write_fasta(
	data     = keep_2150,
	filename = os.path.join(OUT_DATA, f'sequences-2150-keep.fasta')
)

writer.write_fasta(
	data     = drop_2150,
	filename = os.path.join(OUT_DATA, f'sequences-2150-drop.fasta')
)

writer.write_fasta(
	data     = keep_6150,
	filename = os.path.join(OUT_DATA, f'sequences-6150-keep.fasta')
)

writer.write_fasta(
	data     = drop_6150,
	filename = os.path.join(OUT_DATA, f'sequences-6150-drop.fasta')
)

## 3.3 Mutations

In [21]:
# Select random transcripts to mutate

mutation_transcripts = random.choices(list(keep_sequences.keys()), k = 25)
mutation_transcripts = {key : value for key, value in keep_sequences.items() if key in mutation_transcripts}

In [22]:
# Mutate transcripts multiple times

rates = [
	0.01,
	0.05,
	0.10,
	0.15,
	0.25
]

params = {
	'mutation_rate'     : 0.1,
	'insertion_rate'    : 0.0,
	'deletion_rate'     : 0.0,
	'substitution_rate' : 1.0,
	'max_length'        : 6
}

result = mutation_sequence.generate_multi(
	sequences = mutation_transcripts,
	variants  = 20,
	method    = 'random',
	rates     = rates,
	params    = params,
	verbose   = False
)

mutation_sequences = result[0]
mutation_features  = result[1]

  0%|          | 0/125 [00:00<?, ?it/s]

In [23]:
# Merge mutation transcript regions and pad accordingly

mutation_2150, mutation_6150 = feature_extractor.merge_and_pad_sequences(
	sequences = mutation_sequences,
	lengths   = lengths,
	padding   = padding
)

In [24]:
# Compute similiarity betwen orginal and mutated transcript sequences

data_2150 = dict()
data_6150 = dict()

for key, value in mutation_2150.items() :
	splits = key.split(' | ')

	orgkey = splits[0].split('-')[0]
	orgkey = orgkey + ' | ' + ' | '.join(splits[1:])

	orgseq = keep_2150[orgkey]

	match = sum([1 if x == y else 0 for x, y in zip(orgseq, value)])
	match = match / len(orgseq)

	data_2150['{} | {:.5f}'.format(key, match)] = value

for key, value in mutation_6150.items() :
	splits = key.split(' | ')

	orgkey = splits[0].split('-')[0]
	orgkey = orgkey + ' | ' + ' | '.join(splits[1:])

	orgseq = keep_6150[orgkey]

	match = sum([1 if x == y else 0 for x, y in zip(orgseq, value)])
	match = match / len(orgseq)

	data_6150['{} | {:.5f}'.format(key, match)] = value

mutation_2150 = data_2150
mutation_6150 = data_6150

In [25]:
# Save the mutation transcript sequences

writer.write_fasta(
	data     = mutation_2150,
	filename = os.path.join(OUT_DATA, f'mutation-sequence-2150.fasta')
)

writer.write_fasta(
	data     = mutation_6150,
	filename = os.path.join(OUT_DATA, f'mutation-sequence-6150.fasta')
)

In [26]:
# Extract mutation features

mutation_features_frequency = {
	key : numpy.array(value['Frequency'])
	for key, value in mutation_features.items()
}

mutation_features_stability = {
	key : numpy.array(value['Stability'])
	for key, value in mutation_features.items()
}

In [27]:
# Save the mutation features

writer.write_npz(
	data     = mutation_features_frequency,
	filename = os.path.join(OUT_DATA, 'mutation-features-frequency')
)

writer.write_npz(
	data     = mutation_features_stability,
	filename = os.path.join(OUT_DATA, 'mutation-features-stability')
)

In [28]:
# Save merged features

mutation_features_base = dict()

for key in mutation_features_frequency.keys() :
	freq = mutation_features_frequency[key]
	stab = mutation_features_stability[key]

	mutation_features_base[key] = numpy.concatenate((freq, stab), axis = 0)

writer.write_npz(
	data     = mutation_features_base,
	filename = os.path.join(OUT_DATA, 'mutation-features-base')
)

## 3.4 Features

In [29]:
# Extract features

keep_features_frequency = {
	key : numpy.array(value['Frequency'])
	for key, value in keep_features.items()
}

keep_features_stability = {
	key : numpy.array(value['Stability'])
	for key, value in keep_features.items()
}

drop_features_frequency = {
	key : numpy.array(value['Frequency'])
	for key, value in drop_features.items()
}

drop_features_stability = {
	key : numpy.array(value['Stability'])
	for key, value in drop_features.items()
}

In [30]:
# Save the features

writer.write_npz(
	data     = keep_features_frequency,
	filename = os.path.join(OUT_DATA, 'features-frequency-keep')
)

writer.write_npz(
	data     = keep_features_stability,
	filename = os.path.join(OUT_DATA, 'features-stability-keep')
)

writer.write_npz(
	data     = drop_features_frequency,
	filename = os.path.join(OUT_DATA, 'features-frequency-drop')
)

writer.write_npz(
	data     = drop_features_stability,
	filename = os.path.join(OUT_DATA, 'features-stability-drop')
)

In [31]:
# Save merged features

keep_features_base = dict()
drop_features_base = dict()

for key in keep_features_frequency.keys() :
	freq = keep_features_frequency[key]
	stab = keep_features_stability[key]

	keep_features_base[key] = numpy.concatenate((freq, stab), axis = 0)

writer.write_npz(
	data     = keep_features_base,
	filename = os.path.join(OUT_DATA, 'features-base-keep')
)

for key in drop_features_frequency.keys() :
	freq = drop_features_frequency[key]
	stab = drop_features_stability[key]

	drop_features_base[key] = numpy.concatenate((freq, stab), axis = 0)

writer.write_npz(
	data     = drop_features_base,
	filename = os.path.join(OUT_DATA, 'features-base-drop')
)

### 3.5 Anndata

In [32]:
# Save the annotated data with multiple layers

writer.write_h5ad(
	data     = anndata[:, list(features.keys())].copy(),
	filename = os.path.join(OUT_DATA, 'arabidopsis-r36.h5ad')
)